In [1]:
import kagglehub
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler , OneHotEncoder , LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
path = kagglehub.dataset_download("jahnavipaliwal/video-game-reviews-and-ratings")

100%|██████████| 1.10M/1.10M [00:00<00:00, 1.57MB/s]

Extracting files...


In [3]:
print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/jahnavipaliwal/video-game-reviews-and-ratings/versions/3


In [4]:
path ="/root/.cache/kagglehub/datasets/jahnavipaliwal/video-game-reviews-and-ratings/versions/3"

In [5]:
print(os.listdir(path))

['video_game_reviews.csv']


In [6]:
df=pd.read_csv(os.path.join(path,"video_game_reviews.csv"))

,Game Title,User Rating,Age Group Targeted,Price,Platform,Requires Special Device,Developer,Publisher,Release Year,Genre,Multiplayer,Game Length (Hours),Graphics Quality,Soundtrack Quality,Story Quality,User Review Text,Game Mode,Min Number of Players
0,Grand Theft Auto V,36.4,All Ages,41.41,PC,No,Game Freak,Innersloth,2015,Adventure,No,55.3,Medium,Average,Poor,"Solid game, but too many bugs.",Offline,1
1,The Sims 4,38.3,Adults,57.56,PC,No,Nintendo,Electronic Arts,2015,Shooter,Yes,34.6,Low,Poor,Poor,"Solid game, but too many bugs.",Offline,3
2,Minecraft,26.8,Teens,44.93,PC,Yes,Bungie,Capcom,2012,Adventure,Yes,13.9,Low,Good,Average,"Great game, but the graphics could be better.",Offline,5
3,Bioshock Infinite,38.4,All Ages,48.29,Mobile,Yes,Game Freak,Nintendo,2015,Sports,No,41.9,Medium,Good,Excellent,"Solid game, but the graphics could be better.",Online,4
4,Half-Life: Alyx,30.1,Adults,55.49,PlayStation,Yes,Game Freak,Epic Games,2022,RPG,Yes,13.2,High,Poor,Good,"Great game, but too many bugs.",Offline,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47769,Minecraft,41.6,All Ages,49.11,PlayStation,Yes,Valve,Innersloth,2016,Party,No,56.7,Low,Excellent,Average,"Amazing game, but the graphics could be better.",Online,4
47770,The Legend of Zelda: Breath of the Wild,24.2,Kids,31.69,Xbox,Yes,Valve,Electronic Arts,2023,Strategy,No,29.7,High,Good,Excellent,"Solid game, but the graphics could be better.",Offline,1
47771,Animal Crossing: New Horizons,26.7,All Ages,44.90,PlayStation,Yes,Game Freak,Square Enix,2018,Sports,No,8.2,Low,Poor,Good,"Great game, but the gameplay is amazing.",Offline,5
47772,The Legend of Zelda: Breath of the Wild,22.5,Kids,29.99,Xbox,Yes,Epic Games,Epic Games,2018,Simulation,Yes,29.7,High,Poor,Excellent,"Disappointing game, but the graphics could be ...",Offline,1


(47774, 18)

In [7]:
df.duplicated()
# # df.duplicated().sum()

,0
0,False
1,False
2,False
3,False
4,False
...,...
47769,False
47770,False
47771,False
47772,False


In [ ]:
print(df.shape)
print(df.columns)
print(df.isnull().sum())

(47774, 18)
Index(['Game Title', 'User Rating', 'Age Group Targeted', 'Price', 'Platform',
       'Requires Special Device', 'Developer', 'Publisher', 'Release Year',
       'Genre', 'Multiplayer', 'Game Length (Hours)', 'Graphics Quality',
       'Soundtrack Quality', 'Story Quality', 'User Review Text', 'Game Mode',
       'Min Number of Players'],
      dtype='object')
Game Title                 0
User Rating                0
Age Group Targeted         0
Price                      0
Platform                   0
Requires Special Device    0
Developer                  0
Publisher                  0
Release Year               0
Genre                      0
Multiplayer                0
Game Length (Hours)        0
Graphics Quality           0
Soundtrack Quality         0
Story Quality              0
User Review Text           0
Game Mode                  0
Min Number of Players      0
dtype: int64


In [ ]:
print(len(df.columns))
print(df.columns)

18
Index(['Game Title', 'User Rating', 'Age Group Targeted', 'Price', 'Platform',
       'Requires Special Device', 'Developer', 'Publisher', 'Release Year',
       'Genre', 'Multiplayer', 'Game Length (Hours)', 'Graphics Quality',
       'Soundtrack Quality', 'Story Quality', 'User Review Text', 'Game Mode',
       'Min Number of Players'],
      dtype='object')


In [8]:
display(df)

,Game Title,User Rating,Age Group Targeted,Price,Platform,Requires Special Device,Developer,Publisher,Release Year,Genre,Multiplayer,Game Length (Hours),Graphics Quality,Soundtrack Quality,Story Quality,User Review Text,Game Mode,Min Number of Players
0,Grand Theft Auto V,36.4,All Ages,41.41,PC,No,Game Freak,Innersloth,2015,Adventure,No,55.3,Medium,Average,Poor,"Solid game, but too many bugs.",Offline,1
1,The Sims 4,38.3,Adults,57.56,PC,No,Nintendo,Electronic Arts,2015,Shooter,Yes,34.6,Low,Poor,Poor,"Solid game, but too many bugs.",Offline,3
2,Minecraft,26.8,Teens,44.93,PC,Yes,Bungie,Capcom,2012,Adventure,Yes,13.9,Low,Good,Average,"Great game, but the graphics could be better.",Offline,5
3,Bioshock Infinite,38.4,All Ages,48.29,Mobile,Yes,Game Freak,Nintendo,2015,Sports,No,41.9,Medium,Good,Excellent,"Solid game, but the graphics could be better.",Online,4
4,Half-Life: Alyx,30.1,Adults,55.49,PlayStation,Yes,Game Freak,Epic Games,2022,RPG,Yes,13.2,High,Poor,Good,"Great game, but too many bugs.",Offline,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47769,Minecraft,41.6,All Ages,49.11,PlayStation,Yes,Valve,Innersloth,2016,Party,No,56.7,Low,Excellent,Average,"Amazing game, but the graphics could be better.",Online,4
47770,The Legend of Zelda: Breath of the Wild,24.2,Kids,31.69,Xbox,Yes,Valve,Electronic Arts,2023,Strategy,No,29.7,High,Good,Excellent,"Solid game, but the graphics could be better.",Offline,1
47771,Animal Crossing: New Horizons,26.7,All Ages,44.90,PlayStation,Yes,Game Freak,Square Enix,2018,Sports,No,8.2,Low,Poor,Good,"Great game, but the gameplay is amazing.",Offline,5
47772,The Legend of Zelda: Breath of the Wild,22.5,Kids,29.99,Xbox,Yes,Epic Games,Epic Games,2018,Simulation,Yes,29.7,High,Poor,Excellent,"Disappointing game, but the graphics could be ...",Offline,1


In [ ]:
# print(df['Min Number of Players'])

0        1
1        3
2        5
3        4
4        1
        ..
47769    4
47770    1
47771    5
47772    1
47773    8
Name: Min Number of Players, Length: 47774, dtype: int64


In [10]:
numeric_cols= ["User Rating", "Price", "Game Length (Hours)","Min Number of Players"]
binary_cols=["Requires Special Device", "Multiplayer"]
ordinal_cols=[ "Soundtrack Quality","Story Quality"]
# categorical_cols=["Age Group Targeted","Platform","Game Mode","Genre","Developer","Publisher"]
categorical_cols=["Platform","Game Mode"]
text_col="User Review Text"
display_col="Game Title"

In [11]:
# Numeric: fill with median
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# Binary: convert Yes/No -> 1/0, fill NaN as 0
df[binary_cols] = df[binary_cols].fillna("No")
df[binary_cols] = df[binary_cols].applymap(lambda x: 1 if str(x).lower() in ['yes', 'true', '1'] else 0)

# Ordinal: map strings -> numbers, fill missing with median
graphics_mapping = {"Low":1, "Medium":2, "High":3}
story_mapping    = {"Poor":1, "Average":2, "Good":3, "Excellent":4}
sound_mapping    = {"Poor":1, "Average":2, "Good":3, "Excellent":4}

df["Graphics Quality"] = df["Graphics Quality"].map(graphics_mapping)
df["Story Quality"]    = df["Story Quality"].map(story_mapping)
df["Soundtrack Quality"] = df["Soundtrack Quality"].map(sound_mapping)

# Fill NaN in ordinal columns with median
df[ordinal_cols] = df[ordinal_cols].fillna(df[ordinal_cols].median())

# Categorical: fill NaN with 'Unknown'
df[categorical_cols] = df[categorical_cols].fillna("Unknown")

# Text: fill NaN with empty string
df[text_col] = df[text_col].fillna("")

/tmp/ipython-input-2856285876.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[binary_cols] = df[binary_cols].applymap(lambda x: 1 if str(x).lower() in ['yes', 'true', '1'] else 0)


In [12]:
# Label encoding : high cardinality
# high_card_cols=["Developer","Publisher","Genre"]
# for col in high_card_cols:
#   le=LabelEncoder()
#   df[col]=le.fit_transform(df[col])

In [13]:
# low cardinality : one_hot encoding
df1=df.copy()
low_card_cols=["Platform","Game Mode"]
df1=pd.get_dummies(df,columns=low_card_cols)

In [14]:
# Low-cardinality: One-Hot Encoding
# low_card_cols = ["Platform", "Game Mode"]
# df = pd.get_dummies(df, columns=low_card_cols)

In [15]:
scaler = MinMaxScaler()
df[numeric_cols + ordinal_cols] = scaler.fit_transform(df[numeric_cols + ordinal_cols])

In [16]:
# UNSTRUCTURED DATA

df["User Review Text"] = df["User Review Text"].str.lower()
df["User Review Text"] = df["User Review Text"].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))

In [17]:
import spacy
nlp = spacy.load("en_core_web_sm")
df["User Review Text"] = df["User Review Text"].apply(lambda doc: " ".join([token.lemma_ for token in nlp(doc) if not token.is_stop]))

In [18]:
df.head()

,Game Title,User Rating,Age Group Targeted,Price,Requires Special Device,Developer,Publisher,Release Year,Genre,Multiplayer,...,Story Quality,User Review Text,Min Number of Players,Platform_Mobile,Platform_Nintendo Switch,Platform_PC,Platform_PlayStation,Platform_Xbox,Game Mode_Offline,Game Mode_Online
0,Grand Theft Auto V,0.667513,All Ages,0.53550,0,Game Freak,Innersloth,2015,Adventure,0,...,0.000000,solid game bug,0.000000,False,False,True,False,False,True,False
1,The Sims 4,0.715736,Adults,0.93925,0,Nintendo,Electronic Arts,2015,Shooter,1,...,0.000000,solid game bug,0.222222,False,False,True,False,False,True,False
2,Minecraft,0.423858,Teens,0.62350,1,Bungie,Capcom,2012,Adventure,1,...,0.333333,great game graphic well,0.444444,False,False,True,False,False,True,False
3,Bioshock Infinite,0.718274,All Ages,0.70750,1,Game Freak,Nintendo,2015,Sports,0,...,1.000000,solid game graphic well,0.333333,True,False,False,False,False,False,True
4,Half-Life: Alyx,0.507614,Adults,0.88750,1,Game Freak,Epic Games,2022,RPG,1,...,0.666667,great game bug,0.000000,False,False,False,True,False,True,False


In [19]:
df.columns

Index(['Game Title', 'User Rating', 'Age Group Targeted', 'Price',
       'Requires Special Device', 'Developer', 'Publisher', 'Release Year',
       'Genre', 'Multiplayer', 'Game Length (Hours)', 'Graphics Quality',
       'Soundtrack Quality', 'Story Quality', 'User Review Text',
       'Min Number of Players', 'Platform_Mobile', 'Platform_Nintendo Switch',
       'Platform_PC', 'Platform_PlayStation', 'Platform_Xbox',
       'Game Mode_Offline', 'Game Mode_Online'],
      dtype='object')

In [20]:
print(df['User Review Text'])

0                         solid game bug
1                         solid game bug
2                great game graphic well
3                solid game graphic well
4                         great game bug
                      ...               
47769          amazing game graphic well
47770            solid game graphic well
47771        great game gameplay amazing
47772    disappointing game graphic well
47773          amazing game graphic well
Name: User Review Text, Length: 47774, dtype: object


In [21]:
print(len(df.columns))

23


In [22]:
print(df.columns)

Index(['Game Title', 'User Rating', 'Age Group Targeted', 'Price',
       'Requires Special Device', 'Developer', 'Publisher', 'Release Year',
       'Genre', 'Multiplayer', 'Game Length (Hours)', 'Graphics Quality',
       'Soundtrack Quality', 'Story Quality', 'User Review Text',
       'Min Number of Players', 'Platform_Mobile', 'Platform_Nintendo Switch',
       'Platform_PC', 'Platform_PlayStation', 'Platform_Xbox',
       'Game Mode_Offline', 'Game Mode_Online'],
      dtype='object')


In [23]:
from sentence_transformers import SentenceTransformer

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# User input
user_input = "I want a relaxing adventure game with exploration"

# Generate embedding
user_embedding = model.encode([user_input], convert_to_tensor=True)
user_embedding


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tensor([[ 5.5108e-02,  6.6744e-03,  1.0248e-02,  3.4565e-02, -6.2099e-02,
         -4.2031e-02,  5.9379e-02, -6.2281e-02, -5.7517e-02,  4.4735e-02,
         -5.8641e-02, -1.7998e-02, -1.2142e-02,  6.6705e-02, -1.5399e-02,
          2.4657e-02,  7.3914e-02,  6.3284e-02,  1.1441e-01, -1.3860e-02,
         -4.6823e-02,  1.6653e-02, -2.1913e-02, -7.7684e-04, -4.0205e-02,
          3.9429e-02,  2.0153e-02,  1.0622e-02,  4.4254e-02, -6.9691e-02,
          8.5195e-05,  8.7317e-02, -2.7473e-03, -3.7999e-02,  4.8000e-02,
          7.1755e-02, -7.7283e-02, -9.7022e-02,  1.7335e-02, -1.3671e-02,
         -1.0000e-01,  4.6503e-02,  3.7048e-02,  1.0841e-02,  2.5671e-02,
         -4.9307e-03, -5.9381e-02, -1.9402e-02,  9.5441e-02,  6.3347e-02,
          4.7986e-02, -3.9784e-02, -5.3671e-02, -8.5586e-02, -7.1240e-03,
         -4.0763e-02, -6.1997e-02, -3.3570e-02, -7.7891e-03, -6.1998e-03,
          3.7747e-02,  6.2625e-03, -1.5570e-02,  2.4839e-02,  5.6913e-03,
         -3.1581e-03, -1.8596e-02,  6.

In [25]:
import pandas as pd
df=pd.read_csv('/content/video_game_reviews.csv')
df.head()


,Game Title,User Rating,Age Group Targeted,Price,Platform,Requires Special Device,Developer,Publisher,Release Year,Genre,Multiplayer,Game Length (Hours),Graphics Quality,Soundtrack Quality,Story Quality,User Review Text,Game Mode,Min Number of Players
0,Grand Theft Auto V,36.4,All Ages,41.41,PC,No,Game Freak,Innersloth,2015,Adventure,No,55.3,Medium,Average,Poor,"Solid game, but too many bugs.",Offline,1
1,The Sims 4,38.3,Adults,57.56,PC,No,Nintendo,Electronic Arts,2015,Shooter,Yes,34.6,Low,Poor,Poor,"Solid game, but too many bugs.",Offline,3
2,Minecraft,26.8,Teens,44.93,PC,Yes,Bungie,Capcom,2012,Adventure,Yes,13.9,Low,Good,Average,"Great game, but the graphics could be better.",Offline,5
3,Bioshock Infinite,38.4,All Ages,48.29,Mobile,Yes,Game Freak,Nintendo,2015,Sports,No,41.9,Medium,Good,Excellent,"Solid game, but the graphics could be better.",Online,4
4,Half-Life: Alyx,30.1,Adults,55.49,PlayStation,Yes,Game Freak,Epic Games,2022,RPG,Yes,13.2,High,Poor,Good,"Great game, but too many bugs.",Offline,1


In [26]:
df.duplicated().sum()

np.int64(0)

In [46]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch
import os

# class WeightedGameEmbedder:
#     def __init__(self, device=None):
#         self.device = device if device else ('cuda' if torch.cuda.is_available() else 'cpu')
#         print(f"Using device: {self.device}")
#         self.model = SentenceTransformer('all-MiniLM-L6-v2', device=self.device)
#         self.embeddings = {}
#         self.names = []

#     def prepare_and_group(self, df):
#         grouped = df.groupby('Game Title').agg({
#             'Age Group Targeted': lambda x: ' '.join(x.dropna().astype(str)),
#             'Genre': lambda x: ' '.join(x.dropna().astype(str)),
#             'User Review Text': lambda x: ' '.join(x.dropna().astype(str)),
#             'Graphics Quality': lambda x: ' '.join(x.dropna().astype(str))
#         }).reset_index()
#         self.names = grouped['Game Title'].tolist()
#         return grouped

#     def encode_features(self, df, save_dir="embeddings"):
#         os.makedirs(save_dir, exist_ok=True)
#         features = {
#             'title': df['Game Title'].tolist(),
#             'age': df['Age Group Targeted'].tolist(),
#             'genre': df['Genre'].tolist(),
#             'review': df['User Review Text'].tolist(),
#             'graphics': df['Graphics Quality'].tolist(),
#         }
#         for feature, texts in features.items():
#             path = os.path.join(save_dir, f"{feature}_embeddings.npy")
#             if os.path.exists(path):
#                 print(f"Loading {feature} embeddings from {path}")
#                 self.embeddings[feature] = torch.tensor(np.load(path))
#             else:
#                 print(f"Encoding {feature}...")
#                 emb = self.model.encode(
#                     texts,
#                     convert_to_tensor=True,
#                     batch_size=64,
#                     show_progress_bar=True
#                 )
#                 np.save(path, emb.cpu().numpy())
#                 self.embeddings[feature] = emb
#         print("All embeddings ready (loaded or encoded).")

#     def weighted_similarity(self, game_name, weights=None, top_k=5):
#         if weights is None:
#             weights = {'title':0.4, 'age':0.1, 'genre':0.2, 'review':0.2, 'graphics':0.1}

#         if game_name not in self.names:
#             print(f"{game_name} not found")
#             return []

#         idx = self.names.index(game_name)
#         final_sim = np.zeros(len(self.names))
#         for feature, w in weights.items():
#             sims = cosine_similarity(
#                 self.embeddings[feature][idx:idx+1].cpu().numpy(),
#                 self.embeddings[feature].cpu().numpy()
#             )[0]
#             final_sim += w * sims

#         top_idx = final_sim.argsort()[::-1][1:top_k+1]  # skip itself
#         return [(self.names[i], float(final_sim[i])) for i in top_idx]

#     def user_query_similarity(self, user_input, weights=None, top_k=5):
#         """
#         Find top-k games based on a user input query.
#         """
#         if weights is None:
#             weights = {'title':0.4, 'age':0.1, 'genre':0.2, 'review':0.2, 'graphics':0.1}

#         final_sim = np.zeros(len(self.names))

#         # Encode user input **once**
#         user_emb = self.model.encode([user_input], convert_to_tensor=True, batch_size=1, show_progress_bar=False)

#         for feature, w in weights.items():
#             sims = cosine_similarity(
#                 user_emb.cpu().numpy(),
#                 self.embeddings[feature].cpu().numpy()
#             )[0]
#             final_sim += w * sims

#         top_idx = final_sim.argsort()[::-1][:top_k]
#         return [(self.names[i], float(final_sim[i])) for i in top_idx]

# # ---------------------- Usage ----------------------
# # Initialize embedder
# embedder = WeightedGameEmbedder(device='cpu')

# # Prepare grouped dataframe
# grouped_df = embedder.prepare_and_group(df)

# # Encode/load embeddings
# embedder.encode_features(grouped_df, save_dir="embeddings")

# # Find games based on user query
# similar_games = embedder.user_query_similarity("I want a relaxing adventure game with exploration", top_k=5)

# # Print results
# for name, score in similar_games:
#     print(f"{name}: {score:.4f}")


In [47]:
# Initialize the embedder
embedder = WeightedGameEmbedder(device='cpu')  # use 'cpu' if no GPU

# Prepare and group data by game title
grouped_df = embedder.prepare_and_group(df)
embedder.encode_features(grouped_df, save_dir="embeddings")

Using device: cpu
Loading title embeddings from embeddings/title_embeddings.npy
Loading age embeddings from embeddings/age_embeddings.npy
Loading genre embeddings from embeddings/genre_embeddings.npy
Loading review embeddings from embeddings/review_embeddings.npy
Loading graphics embeddings from embeddings/graphics_embeddings.npy
All embeddings ready (loaded or encoded).


In [41]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 29.4 MB/s eta 0:00:00


In [42]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize
import torch
import os
import faiss

class GameRecommender:
    def __init__(self, model_name='all-mpnet-base-v2', device=None, embedding_dir="embeddings"):
        """
        Game recommender using combined embeddings.
        """
        self.device = device if device else ('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")
        self.model = SentenceTransformer(model_name, device=self.device)
        self.embedding_dir = embedding_dir
        os.makedirs(self.embedding_dir, exist_ok=True)
        self.game_embeddings = None
        self.game_names = None
        self.index = None

    def prepare_text(self, df):
        """
        Combine all textual features into a single string per game.
        """
        combined = (
            df['Game Title'].astype(str) + " | " +
            df['Genre'].astype(str) + " | " +
            df['User Review Text'].astype(str) + " | " +
            "Age: " + df['Age Group Targeted'].astype(str) + " | " +
            "Graphics: " + df['Graphics Quality'].astype(str)
        )
        return combined

    def encode_games(self, df):
        """
        Encode all games and save/load embeddings.
        """
        self.game_names = df['Game Title'].tolist()
        embedding_path = os.path.join(self.embedding_dir, "game_embeddings.npy")

        if os.path.exists(embedding_path):
            print(f"Loading game embeddings from {embedding_path}")
            self.game_embeddings = np.load(embedding_path)
        else:
            print("Encoding game embeddings...")
            combined_texts = self.prepare_text(df).tolist()
            self.game_embeddings = self.model.encode(
                combined_texts,
                convert_to_tensor=False,
                batch_size=64,
                show_progress_bar=True
            )
            # Normalize embeddings
            self.game_embeddings = normalize(self.game_embeddings)
            np.save(embedding_path, self.game_embeddings)
            print(f"Saved embeddings to {embedding_path}")

        # Build FAISS index
        dim = self.game_embeddings.shape[1]
        self.index = faiss.IndexFlatIP(dim)  # inner product on normalized = cosine similarity
        self.index.add(self.game_embeddings.astype('float32'))
        print("FAISS index built.")

    def query(self, user_input, top_k=5):
        """
        Find top-k games based on a user query.
        """
        user_emb = self.model.encode([user_input], convert_to_tensor=False)
        user_emb = normalize(user_emb)
        D, I = self.index.search(user_emb.astype('float32'), k=top_k)
        results = [(self.game_names[idx], float(score)) for idx, score in zip(I[0], D[0])]
        return results

# ---------------- Usage Example ----------------
# Load your game dataframe
# df should have columns: ['Game Title', 'Genre', 'User Review Text', 'Age Group Targeted', 'Graphics Quality']
# Example: df = pd.read_csv("games.csv")

# Initialize recommender
recommender = GameRecommender(device='cpu')  # change to 'cuda' if GPU available

# Encode/load game embeddings
recommender.encode_games(df)

# User query
user_query = "I want a relaxing adventure game with exploration"

# Get top 5 similar games
top_games = recommender.query(user_query, top_k=5)

# Print results
for name, score in top_games:
    print(f"{name}: {score:.4f}")

Using device: cpu


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Encoding game embeddings...


Batches:   0%|          | 0/747 [00:00<?, ?it/s]

Saved embeddings to embeddings/game_embeddings.npy
FAISS index built.
Sid Meier’s Civilization VI: 0.5767
1000-Piece Puzzle: 0.5720
Animal Crossing: New Horizons: 0.5719
Sid Meier’s Civilization VI: 0.5714
1000-Piece Puzzle: 0.5701


In [43]:
def encode_games(self, df):
        """
        Encode all games and save/load embeddings.
        """
        self.game_names = df['Game Title'].tolist()
        embedding_path = os.path.join(self.embedding_dir, "game_embeddings.npy")

        if os.path.exists(embedding_path):
            print(f"Loading game embeddings from {embedding_path}")
            self.game_embeddings = np.load(embedding_path)
        else:
            print("Encoding game embeddings...")
            combined_texts = self.prepare_text(df).tolist()
            self.game_embeddings = self.model.encode(
                combined_texts,
                convert_to_tensor=False,
                batch_size=64,
                show_progress_bar=True
            )
            # Normalize embeddings
            self.game_embeddings = normalize(self.game_embeddings)
            np.save(embedding_path, self.game_embeddings)
            print(f"Saved embeddings to {embedding_path}")

        # Build FAISS index
        dim = self.game_embeddings.shape[1]
        self.index = faiss.IndexFlatIP(dim)  # inner product on normalized = cosine similarity
        self.index.add(self.game_embeddings.astype('float32'))
        print("FAISS index built.")



In [44]:
# ---------------- Usage Example ----------------
# Load your game dataframe
# df should have columns: ['Game Title', 'Genre', 'User Review Text', 'Age Group Targeted', 'Graphics Quality']
# Example: df = pd.read_csv("games.csv")

# Initialize recommender
recommender = GameRecommender(device='cpu')  # change to 'cuda' if GPU available

# Encode/load game embeddings
recommender.encode_games(df)

# User query
user_query = "I want a relaxing adventure game with exploration"

# Get top 5 similar games
top_games = recommender.query(user_query, top_k=5)

# Print results
for name, score in top_games:
    print(f"{name}: {score:.4f}")


Using device: cpu
Loading game embeddings from embeddings/game_embeddings.npy
FAISS index built.
Sid Meier’s Civilization VI: 0.5767
1000-Piece Puzzle: 0.5720
Animal Crossing: New Horizons: 0.5719
Sid Meier’s Civilization VI: 0.5714
1000-Piece Puzzle: 0.5701
